In [99]:
setwd("/Users/gimdong-wan/Desktop/김동완 수업자료/통계 데이터베이스")

In [100]:
library("DBI")
library("RSQLite")

In [101]:
# Customer 데이터베이스 연결
conn = dbConnect(SQLite(), dbname = "RSQL/customer.sqlite")

In [102]:
##### PRAGMA statement : SQLite에 특정된 SQL 확장
### sqlite에서 외래 키를 설정하기 위해서는 다음 문장을 실행해야 함
dbExecute(conn, "PRAGMA foreign_keys=ON")

[1] 0

In [103]:
sql = "CREATE TABLE IF NOT EXISTS Customer_Info
( CID TEXT PRIMARY KEY, Name TEXT,
Address TEXT, Tel TEXT, BDay TEXT, SID TEXT,
Job TEXT,
Sex TEXT, Married TEXT )"


In [104]:
rs = dbSendQuery(conn, sql)

In [105]:
# rs 제거
dbClearResult(rs)

In [106]:
# 테이블이 생성되었는지(존재하는지) 확인 
dbExistsTable(conn, "Customer_Info")

[1] TRUE

In [112]:
# Customer_Score 테이블 생성하기
sql = "CREATE TABLE IF NOT EXISTS Customer_Score
( CID TEXT,
Year TEXT not null,
On_Score INTEGER default 0,
Off_Score INTEGER default 0,
PRIMARY KEY (CID, Year),
FOREIGN KEY(CID) REFERENCES Customer_Info(CID) )"
rs = dbSendQuery(conn, sql)
# rs 제거 
dbClearResult(rs)
# 테이블이 생성되었는지(존재하는지) 확인 
dbExistsTable(conn, "Customer_Score")

[1] TRUE

In [108]:
# Customer_Score 테이블 삭제하기
rs = dbSendQuery(conn, "DROP TABLE Customer_Score")
# rs 제거 
dbClearResult(rs)
# 테이블이 존재하는지(삭제되었는지) 확인 
dbExistsTable(conn, "Customer_Score")


[1] FALSE

In [109]:
# Customer_Info 테이블에 데이터 insert 
sql = " INSERT INTO Customer_Info
VALUES ('100001', '김민지', '서울시 광진구', '423-8627', '1975-07-01', '750701-2534218', '공무원', '1', '0') "
rs = dbSendQuery(conn, sql) 
dbClearResult(rs)

In [110]:
# Customer_Info 테이블에 데이터 삽입 결과 확인 
dbGetQuery(conn, "SELECT * FROM Customer_Info")

CID,Name,Address,Tel,BDay,SID,Job,Sex,Married
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100001,김민지,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0


In [113]:
# Customer_Score 테이블에 데이터 insert 
sql = " INSERT INTO Customer_Score VALUES ('100001', '2012', 42, 111) " 
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [114]:
sql = " INSERT INTO Customer_Score VALUES ('100001', '2013', 58, 89) "
rs = dbSendQuery(conn, sql) 
dbClearResult(rs)

In [115]:
 # Customer_Score 테이블에 데이터 삽입 결과 확인 
dbGetQuery(conn, "SELECT * FROM Customer_Score ")


CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111
100001,2013,58,89


In [118]:
# Customer_Info 테이블에 있는 데이터 update
sql = "UPDATE Customer_Info SET Address = '서울시 영등포구'
    WHERE CID = '100001' " 
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [119]:
# Customer_Info 테이블에 있는 데이터 갱신 결과 확인 
dbGetQuery(conn, "SELECT * FROM Customer_Info")

CID,Name,Address,Tel,BDay,SID,Job,Sex,Married
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100001,김민지,서울시 영등포구,423-8627,1975-07-01,750701-2534218,공무원,1,0


In [120]:
# Customer_Score 테이블에 있는 데이터 update
sql = "UPDATE Customer_Score SET Off_Score = 98
WHERE CID = '100001' and Year='2013' " 
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [121]:
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111
100001,2013,58,98


In [122]:
# Customer_Score 테이블에 있는 특정 데이터 delete 
sql = "DELETE FROM Customer_Score
    WHERE CID = '100001' and Year='2013' " 
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [123]:
# Customer_Score 테이블에 있는 특정 데이터 삭제 결과 확인 
dbGetQuery(conn, "SELECT * FROM Customer_Score")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111


In [124]:
# Customer_Score 테이블에 있는 전체 데이터 delete 
sql = "DELETE FROM Customer_Score "
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [125]:
# Customer_Score 테이블에 있는 전체 데이터 삭제 결과 확인 
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>


In [128]:
# Customer 데이터베이스 연결
conn = dbConnect(SQLite(), dbname = "RSQL/Customer.sqlite")

In [129]:
dbListTables(conn)

[1] "Customer_Info"  "Customer_Score"

In [131]:
customer_data = read.csv('RSQL/customer_data.csv',fileEncoding = "CP949", encoding = "UTF-8",header=T)

In [132]:
# CSV 화일의 데이터를 Customer_Info 테이블에 삽입하기
dbWriteTable(conn = conn, name = "Customer_Info", customer_data,
overwrite=T, row.names=FALSE)

In [133]:
# Customer_Info 테이블에 있는 데이터 확인하기 
dbReadTable(conn, "Customer_Info")

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [135]:
# CSV 화일의 데이터를 Customer_Score 테이블에 삽입하기
customer_score = read.csv('RSQL/customer_score.csv')
dbWriteTable(conn = conn, name = "Customer_Score", customer_score,
overwrite=T, row.names=FALSE)

In [136]:
 # Customer_Score 테이블에 있는 데이터 확인하기 
dbReadTable(conn, "Customer_Score")


CID,Year,On_Score,Off_Score
<int>,<int>,<int>,<int>
100001,2012,42,111
100002,2012,12,30
100003,2012,86,128
100004,2012,0,87
100005,2012,122,56
100006,2012,98,17
100007,2012,0,0
100008,2012,56,172
100009,2012,24,45


In [137]:
sql = "SELECT CID, Name, Job FROM Customer_Info" 
dbGetQuery(conn, sql)

CID,Name,Job
<int>,<chr>,<chr>
100001,김민주,공무원
100002,정영미,회사원
100003,최소영,학생
100004,박진재,공무원
100005,이순석,회사원
100006,박선희,회사원
100007,김화연,학생
100008,박민선,학생
100009,노상수,공무원


In [138]:
sql = "SELECT all * FROM Customer_Info" 
dbGetQuery(conn, sql)

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [140]:
sql = "SELECT * from Customer_Info"
dbGetQuery(conn, sql)

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [141]:
sql = "SELECT CID, Name, Address, Tel, Bday, SID, Job, Sex, Married FROM
    Customer_Info"
dbGetQuery(conn, sql)

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [142]:
sql <- "SELECT DISTINCT Job FROM Customer_Info" 
dbGetQuery(conn, sql)

Job
<chr>
공무원
회사원
학생
주부


In [143]:
sql <- "SELECT DISTINCT Job, Sex FROM Customer_Info" 
dbGetQuery(conn, sql)

Job,Sex
<chr>,<int>
공무원,1
회사원,1
학생,1
공무원,0
회사원,0
주부,1


In [144]:
sql = "SELECT CID, Name, Job, Married FROM Customer_Info
    WHERE Job = '회사원' " 
dbGetQuery(conn, sql)

CID,Name,Job,Married
<int>,<chr>,<chr>,<int>
100002,정영미,회사원,1
100005,이순석,회사원,0
100006,박선희,회사원,0
100010,이민석,회사원,1


In [145]:
sql = "SELECT CID, Name, Job, Married FROM Customer_Info
    WHERE Job != '회사원' " 
dbGetQuery(conn, sql)

CID,Name,Job,Married
<int>,<chr>,<chr>,<int>
100001,김민주,공무원,0
100003,최소영,학생,0
100004,박진재,공무원,1
100007,김화연,학생,0
100008,박민선,학생,0
100009,노상수,공무원,1
100011,정영미,주부,1


In [146]:
sql = "SELECT CID, Name, Sex, Married FROM Customer_Info
    WHERE Sex='1' and Married='1' " 
dbGetQuery(conn, sql)

CID,Name,Sex,Married
<int>,<chr>,<int>,<int>
100002,정영미,1,1
100011,정영미,1,1


In [147]:
sql = "SELECT CID, Year, On_Score FROM Customer_Score
    WHERE Year=2013 and On_Score>=50 and On_Score<=100 "
dbGetQuery(conn, sql)

CID,Year,On_Score
<int>,<int>,<int>
100001,2013,58
100002,2013,98
100006,2013,66


In [148]:
sql = "SELECT CID, Year, On_Score+Off_Score TotalScore FROM Customer_Score
    WHERE Year=2013 " 
dbGetQuery(conn, sql)

CID,Year,TotalScore
<int>,<int>,<int>
100001,2013,147
100002,2013,138
100003,2013,530
100004,2013,74
100005,2013,488
100006,2013,78
100007,2013,5
100008,2013,142
100009,2013,8


In [149]:
sql = "SELECT CID, Year, On_Score+Off_Score TotalScore FROM Customer_Score
    WHERE Year=2013 and TotalScore>100 "
dbGetQuery(conn, sql)

CID,Year,TotalScore
<int>,<int>,<int>
100001,2013,147
100002,2013,138
100003,2013,530
100005,2013,488
100008,2013,142
